In [6]:
# DRO actual data 

In [8]:
IN_COLAB = False
REMOUNT = False

In [10]:
# RUN IN COLAB ONLY: mount Google drive
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [12]:
# RUN IN COLAB ONLY: unmount, remount Google drive - if required
if REMOUNT:
    !fusermount -u /content/drive 2>/dev/null || true
    !rm -rf /content/drive
    from google.colab import auth
    auth.authenticate_user()
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

In [14]:
# RUN IN COLAB ONLY: force-fetch latest files from GitHub (overwrite local copies)
if IN_COLAB:
    !rm -rf /content/SLDS
    !git clone --depth=1 https://github.com/chrismader1/SLDS.git /content/SLDS
    import sys
    sys.path.append("/content/SLDS")

In [16]:
# RUN IN COLAB ONLY: prevent timeouts
if IN_COLAB:
    from IPython.display import Javascript, display
    display(Javascript("""
    (function keepAlive(){
      function clickConnect(){
        // Try shadow-root button (newer Colab)
        const el = document.querySelector('colab-connect-button');
        if (el && el.shadowRoot){
          const btn = el.shadowRoot.querySelector('#connect');
          if (btn){ btn.click(); console.log('keepAlive: clicked shadow connect'); return; }
        }
        // Fallbacks
        const btn2 = document.querySelector('#connect, button#connect');
        if (btn2){ btn2.click(); console.log('keepAlive: clicked #connect'); return; }
        console.log('keepAlive: connect button not found');
      }
      setInterval(clickConnect, 60 * 1000);
      console.log('keepAlive: armed');
    })();
    """))

In [18]:
# RUN IN COLAB ONLY: clear cache
if IN_COLAB:
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_results.csv"
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_segments.csv"
    !echo "Reset done."

In [20]:
if IN_COLAB:
    !pip -q install "cupy-cuda12x>=13.0.0"

In [22]:
if IN_COLAB:
    import cupy as cp
    assert cp.cuda.runtime.getDeviceCount() > 0
    print(cp.cuda.runtime.getDeviceProperties(0)['name'])

In [24]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import itertools, warnings, os
from dro import *

# import warnings
# warnings.filterwarnings("ignore", message=r"Argument (sub|subj) .* Incorrect array format causing data to be copied")
# pd.set_option('future.no_silent_downcasting', True)

GPU=False


In [25]:
# paths

res = "gridsearch_results1"
par = "gridsearch_segments1"

COLAB_PATHS = {
    "data_excel": "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/bbg_data.xlsx",  # Google Drive
    "ff_dir": "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/",                   # Google Drive
    "ff_files": {
        "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
        "ff3": "F-F_Research_Data_Factors_daily.csv",
        "mom": "F-F_Momentum_Factor_daily.csv",},
    "results_csv": f"/content/drive/MyDrive/Colab Notebooks/SLDS/Out/{res}.csv",            # Google Drive
    "segments_parquet": f"/content/drive/MyDrive/Colab Notebooks/SLDS/Out/{par}.parquet",  # Google Drive
    "tmp_dir":          "/content/tmp_slds/",                      # Colab local
    "segments_tmp_csv": "/content/tmp_slds/segments_tmp.csv",      # Colab local
}

LOCAL_PATHS = {
    "data_excel": "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx",
    "ff_dir": "/Users/chrismader/Python/SLDS/Data/",
    "ff_files": {
        "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
        "ff3": "F-F_Research_Data_Factors_daily.csv",
        "mom": "F-F_Momentum_Factor_daily.csv",},
    "results_csv": f"/Users/chrismader/Python/SLDS/Output/{res}.csv",
    "segments_parquet": f"/Users/chrismader/Python/SLDS/Output/{par}.parquet",
    "tmp_dir":          "/Users/chrismader/Python/SLDS/tmp_slds/",
    "segments_tmp_csv": "/Users/chrismader/Python/SLDS/tmp_slds/segments_tmp.csv",
}

In [26]:
# -------------------------
# CONFIG
# -------------------------

CONFIG = {
    # rSLDS model preferences
    "prefer_configs": [
        # {'config': '[g,v,h]',        'n_regimes': 3, 'dim_latent': 2},
        # {'config': '[g,v]',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': '[y,g,v,h]',      'n_regimes': 3, 'dim_latent': 3},
        # {'config': '[y,h]',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': '[y]',            'n_regimes': 3, 'dim_latent': 1},
        # {'config': 'factor1',        'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'factor1_vix',    'n_regimes': 3, 'dim_latent': 3},
        {'config': 'factor2_ff3',    'n_regimes': 3, 'dim_latent': 3},
        # {'config': 'factor2_ff3mom', 'n_regimes': 3, 'dim_latent': 4},
        # {'config': 'factor2_ff5',    'n_regimes': 3, 'dim_latent': 5},
        # {'config': 'factor2_ff5mom', 'n_regimes': 3, 'dim_latent': 6},
        # {'config': 'fund1',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'fund1_vix',      'n_regimes': 3, 'dim_latent': 3},
        # {'config': 'fund2',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'fund2_vix',      'n_regimes': 3, 'dim_latent': 3},
        # {'config': 'fund3',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'fund3_vix',      'n_regimes': 3, 'dim_latent': 3},
    ],

    # Optimizer
    "seed": None,
    "start_dt": None,
    "end_dt": None,
    "min_assets": 3,
    "GLOBAL": {"risk_budget": 0.30, "risk_free_rate": 0.0, "epsilon_sigma": 1e-6},
    "delta_name": "bootstrap_np",

    # Delta defaults (flat under CONFIG)
    "delta_defaults": {
        "kappa_l2":           {"delta_method": "kappa_l2",           "kappa": 1.0},
        "kappa_rate":         {"delta_method": "kappa_rate",         "kappa": 1.0},
        "bound_ek":           {"delta_method": "bound_ek",           "alpha": 0.05, "c1": 3.0, "c2": 1.0, "a": 2.0},
        "bootstrap_np":       {"delta_method": "bootstrap_np",       "alpha": 0.05, "B": 100, "seed": 0},
        "bootstrap_gaussian": {"delta_method": "bootstrap_gaussian", "alpha": 0.05, "B": 100, "seed": 0},
    },}


PATHS = COLAB_PATHS if IN_COLAB else LOCAL_PATHS
for k, v in PATHS.items(): 
    CONFIG[k] = v
# per-security temp file templates used by IOManager
CONFIG["tmp_results_fmt"]  = "{tmp_dir}/tmp_res_{security}.csv"
CONFIG["tmp_segments_fmt"] = "{tmp_dir}/tmp_seg_{security}.csv"


In [ ]:
# -------------------------
# Execute
# -------------------------

# filename_results = '/Users/chrismader/Python/SLDS/Output/gridsearch_results.csv'
# res = pd.read_csv(filename_results)
# securities = list(res.security.unique())
# securities = ['NVDA', 'AAPL', 'AVGO', 'GOOGL', 'ORCL', 'IBM', 'CSCO', 'CRM']
securities = None

out = dro_pipeline(securities, CONFIG, verbose=True)
save_out(out, "/Output/dro_run.pkl.gz")

/opt/anaconda3/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/opt/anaconda3/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/opt/anaconda3/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


[AAPL] raw segments: [0, 1308, 1569, 1831, 2092, 2353, 2613, 2734, 2735, 2833, 2834, 2835, 2875, 2893, 2923, 2957, 2967, 3079, 3105, 3137, 3400, 3463, 3464, 3521]
[ACN] raw segments: [0, 1308, 1569, 1831, 2092, 2353, 2367, 2370, 2388, 2389, 2391, 2442, 2443, 2587, 2588, 2613, 2834, 2835, 2893, 2897, 2919, 2923, 2958, 2964, 3080, 3096, 3137, 3400, 3463, 3464, 3521]
[ADBE] raw segments: [0, 1308, 1569, 1831, 2353, 2613, 3137, 3209, 3255, 3273, 3283, 3284, 3287, 3383, 3400, 3463, 3464, 3521]
[AMAT] raw segments: [0, 1308, 1831, 2125, 2127, 2128, 2130, 2135, 2137, 2138, 2141, 2144, 2147, 2148, 2149, 2155, 2156, 2166, 2167, 2875, 3137, 3272, 3274, 3277, 3279, 3285, 3287, 3308, 3352, 3355, 3382, 3384, 3521]
[AMD] raw segments: [0, 1308, 1569, 2092, 2140, 2141, 2148, 2150, 2353, 2613, 3209, 3273, 3283, 3284, 3285, 3384, 3400, 3463, 3464, 3521]
[AMZN] raw segments: [0, 1569, 1831, 2135, 2137, 2138, 2142, 2367, 2372, 2388, 2391, 2442, 2445, 2490, 2492, 2535, 2537, 2589, 2613, 2875, 3137, 3209, 

/opt/anaconda3/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/opt/anaconda3/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/opt/anaconda3/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


[t=1569 | 2018-01-05] active: ['AAPL', 'ACN', 'ADBE', 'AMAT', 'AMD', 'AMZN', 'AVGO', 'BKNG', 'COST', 'CRM', 'CSCO', 'DIS', 'GOOGL', 'HD', 'IBM', 'INTU', 'KO', 'MCD', 'META', 'MSFT', 'NFLX', 'NOW', 'NVDA', 'ORCL', 'PG', 'PM', 'QCOM', 'T', 'TSLA', 'TXN', 'VZ', 'WMT']
